## 数据读取

In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import sklearn
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import jieba
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']

D:\Program Files\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.1.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
data_path = "../data/data.csv"
stopword_path = "../data/stop_words.txt"
data = pd.read_csv(data_path)
data.head()

,类别名字,文本数据
0,娱乐,黎明喜当爹满面春风，乐基儿发福成大妈！离婚后两人差距这么大？
1,旅游,亚洲排名第一都市，北京GDP不及它一半，上海10年都赶超不了
2,国际,悠仁：日本明仁天皇孙辈唯一男丁堪称“独苗”，将来可能继位
3,娱乐,被意大利炮干掉的阿部规秀到底是个什么官？
4,财经,公司亏损时，股东如何维护自己的利益？


In [3]:
# 去除证券类
data = data[data["类别名字"] != "证券"]

## 分词去停用词

In [4]:
with open(stopword_path, "r", encoding="utf-8") as f:
    stopwords = f.readlines()

stopwords = [i.strip() for i in stopwords]

def segment(data):
    data = jieba.cut(data)
    data = [i for i in data if i not in stopwords]
    return data

In [5]:
data["分词后的数据"] = data["文本数据"].apply(segment)
data.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\jtsun\AppData\Local\Temp\jieba.cache
Loading model cost 0.803 seconds.
Prefix dict has been built succesfully.


,类别名字,文本数据,分词后的数据
0,娱乐,黎明喜当爹满面春风，乐基儿发福成大妈！离婚后两人差距这么大？,"[黎明, 喜当爹, 满面春风, 乐基儿, 发福, 成, 大妈, 离婚, 两人, 差距, 大]"
1,旅游,亚洲排名第一都市，北京GDP不及它一半，上海10年都赶超不了,"[亚洲, 排名, 第一, 都市, 北京, GDP, 不及, 它, 一半, 上海, 10, 年..."
2,国际,悠仁：日本明仁天皇孙辈唯一男丁堪称“独苗”，将来可能继位,"[悠仁, 日本, 明, 仁天皇, 孙辈, 唯一, 男丁, 堪称, 独苗, 将来, 可能, 继位]"
3,娱乐,被意大利炮干掉的阿部规秀到底是个什么官？,"[被, 意大利, 炮, 干掉, 阿部规秀, 到底, 是, 个, 什么, 官]"
4,财经,公司亏损时，股东如何维护自己的利益？,"[公司, 亏损, 时, 股东, 如何, 维护, 自己, 利益]"


In [6]:
data_all = data["分词后的数据"].tolist()
print(np.shape(data_all))
print(data_all[0])

(294304,)
['黎明', '喜当爹', '满面春风', '乐基儿', '发福', '成', '大妈', '离婚', '两人', '差距', '大']


In [7]:
def get_word2vec(sentences,fname,min_count_num,embedding_dim):
    if not os.path.exists(fname):
        model = Word2Vec(sentences, min_count=min_count_num, size=embedding_dim)
        model.save(fname)
    else:
        model = Word2Vec.load(fname)
    
    unk_vector = np.zeros(shape=[embedding_dim,])
    
    data_tensor = []
    for sent in sentences:        
        sent_matrix = []
        for word in sent:
            try:
                temp_vector = model[word]
            except KeyError:
                temp_vector = unk_vector
            sent_matrix.append(temp_vector)
        sent_matrix = np.mean(sent_matrix,axis=0)
        data_tensor.append(sent_matrix)
        
    return np.array(data_tensor)

In [8]:
embedding_dim = 50
min_count_num=5
fname = '../data/word2vec.model'
data_all = get_word2vec(data_all,fname,min_count_num,embedding_dim=embedding_dim)
print(np.shape(data_all))

D:\Program Files\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(294304, 50)


静待训练词向量……

## 模型

In [9]:
label_all = data["类别名字"].tolist()

def change_label(data):
    dict_all = {}
    set_all = list(set(data))
    for i,content in enumerate(set_all):
        dict_all[content] = i
    return dict_all
dict_all = change_label(label_all)
label_id = [dict_all[i] for i in label_all]

In [10]:
print(np.shape(data_all))
print(np.shape(label_id))
print(dict_all)

(294304, 50)
(294304,)
{'体育': 0, '教育': 12, '军事': 2, '房产': 3, '旅游': 6, '民生': 7, '财经': 8, '科技': 9, '电竞': 10, '汽车': 5, '文化': 11, '国际': 1, '娱乐': 13, '农业': 4}


## 训练集和测试集的划分

In [11]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_label,test_label = train_test_split(data_all,label_id,train_size=0.8,test_size=0.2)
print(np.shape(train_data))
print(np.shape(test_data))
print(np.shape(train_label))
print(np.shape(test_label))

(235443, 50)
(58861, 50)
(235443,)
(58861,)


## 模型部分

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

model = LogisticRegression()
model.fit(train_data,train_label)
pred_result = model.predict(test_data)

train_pred_result = model.predict(train_data)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [13]:
print("训练集结果：")
accuracy = accuracy_score(y_pred=train_pred_result,y_true=train_label)
precision = np.average(precision_score(y_pred=train_pred_result,y_true=train_label,average=None))
recall = np.average(recall_score(y_pred=train_pred_result,y_true=train_label,average=None))
f1score = np.average(f1_score(y_pred=train_pred_result,y_true=train_label,average=None))
print("准确率：",accuracy)
print("紧缺率：",precision)
print("召回率：",recall)
print("f1值：",f1score)

print("测试集结果：")
accuracy = accuracy_score(y_pred=pred_result,y_true=test_label)
precision = np.average(precision_score(y_pred=pred_result,y_true=test_label,average=None))
recall = np.average(recall_score(y_pred=pred_result,y_true=test_label,average=None))
f1score = np.average(f1_score(y_pred=pred_result,y_true=test_label,average=None))
print("准确率：",accuracy)
print("紧缺率：",precision)
print("召回率：",recall)
print("f1值：",f1score)


训练集结果：
准确率： 0.7021189842127394
紧缺率： 0.6902410995433323
召回率： 0.6709492466380821
f1值： 0.6787378012150175
测试集结果：
准确率： 0.6988668218344914
紧缺率： 0.686264148358806
召回率： 0.6680965871272807
f1值： 0.6752743529695983


## 测试

In [19]:
input_data = input("请输入待分类的文本：")

with open(stopword_path,"r",encoding="utf-8") as f:
    stopwords = f.readlines()

stopwords = [i.strip() for i in stopwords]

def segment(data):
    data = list(jieba.cut(data))
    data = [i for i in data if i not in stopwords]
    return data


def get_sent_word2vec(sent,seq_length=30):
    
    seg_data = segment(sent)
    word2vec_model = Word2Vec.load("../data/word2vec.model")
    
    unk_vector = np.zeros(shape=[embedding_dim,])
    sent_matrix = []
    for word in sent:
        try:
            temp_vector = word2vec_model[word]
        except KeyError:
            temp_vector = unk_vector
        sent_matrix.append(temp_vector)
    tensor = np.reshape(np.mean(sent_matrix,axis=0),(1,embedding_dim))
    return tensor
    


data_vector = get_sent_word2vec(input_data)

pred_result = model.predict(data_vector)
dict_all_reverse = dict(zip(dict_all.values(),dict_all.keys()))
print("当前文本类别为:",dict_all_reverse[pred_result[0]])

请输入待分类的文本：海边看日落
当前文本类别为: 旅游


D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
